In [1]:
import numpy as np

In [2]:
c=[[0 for x in range(21)] for x in range(21)]

In [3]:
def connect(c,a,b):
    c[a][b]=c[b][a]=1

In [4]:
connect(c,0,9)
connect(c,1,18)
connect(c,2,13)
connect(c,1,7)
connect(c,4,10)
connect(c,4,20)
connect(c,7,8)
connect(c,9,16)
connect(c,11,17)
connect(c,17,18)
connect(c,18,19)
connect(c,12,18)
connect(c,6,12)
connect(c,9,10)
connect(c,18,13)
connect(c,14,15)
connect(c,19,13)
connect(c,19,1)
connect(c,3,10)

In [5]:
active=np.array([0,1,2,4,9,10,13,17,18,19])
con=np.array(c)

In [16]:
(con.sum(axis=1)==1).astype(np.float64).shape
active_index=np.array((1,1,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,1,1,0))
active_index.shape

(21,)

In [24]:
%%timeit
red=con[active][:,active]

2.82 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
red

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0]])

In [25]:
%%timeit
(con*active_index).T*active_index

3.26 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


```python
def get_new_neighbours(red, index, l):
    neigh=np.arange(0,red.shape[0], 1)[red[index].astype(np.bool8)]
    mask=np.isin(neigh, l, invert=True)
    return neigh[mask]
```

```python
def get_families(red):
    n_active=len(red)
    n_fam=-1
    l=[]
    ll=[]
    
    def get_new_neighbours(red, index, l):
        neigh=np.arange(0,red.shape[0], 1)[red[index].astype(np.bool8)]
        mask=np.isin(neigh, l, invert=True)
        return neigh[mask]
    
    def save(neigh, fam, ll, l):
        ll[fam]=ll[fam]+ neigh.tolist()
        l=l+neigh.tolist()
        print("saving")
        print("--> ll ", ll)
        print("--> l ", l)
        return ll,l
    
    def get_active_connected(red,i, ll, l, n_fam):
        new_neigh=get_new_neighbours(red, i, l)
        print("got neighbours of node",i)
        print("--> new_neigh:",new_neigh)
        if new_neigh.size!=0:
            print("not an empty list, now saving")
            ll,l=save(new_neigh, n_fam, ll, l)
            print("begin loop on ", new_neigh)
            for j in new_neigh:
                print("get all active connections of ", j)
                ll,l=get_active_connected(red, j, ll, l, n_fam)
        else:
            print("empty list")
        return ll, l
    
    for i in range(n_active):
        print("trying ", i)
        if i not in l:
            print("begin new family")
            print("get all active connections of ", i)
            n_fam+=1
            ll.append([])
            ll[n_fam].append(i)
            l.append(i)
            ll,l=get_active_connected(red,i, ll, l, n_fam) 
        else:
            print("already in l")
    return ll,l
```

## Mixed python code

In [6]:
def get_families_py(red):
    n_active=len(red)
    n_fam=-1
    l=[]
    lll=[]
    
    def get_new_neighbours(red, index, l):
        neigh=np.arange(0,red.shape[0], 1)[red[index].astype(np.bool8)]
        mask=np.isin(neigh, l, invert=True)
        return neigh[mask]
    
    def save(neigh, n_fam,  l,lll):
        l=l+neigh.tolist()
        lll[n_fam]+=neigh.size
        return l,lll
    
    def get_active_connected(red,i,  l,lll, n_fam):
        new_neigh=get_new_neighbours(red, i, l)
        if new_neigh.size!=0:
            l,lll=save(new_neigh, n_fam, l,lll)
            for j in new_neigh:
                l,lll=get_active_connected(red, j, l,lll, n_fam)
        return l,lll
    
    for i in range(n_active):
        if i not in l:
            n_fam+=1
            l.append(i)
            lll.append(1)
            l,lll=get_active_connected(red, i, l, lll, n_fam) 
    return sorted(lll, reverse=True)

## Pure numpy

In [7]:
def get_families_np(red):
    n_active=red.shape[0]
    n_fam=-1
    l=np.array((),dtype=np.int16)
    lll=np.array((),dtype=np.int16)
    
    def get_new_neighbours(red, index, l):
        neigh=np.arange(0,red.shape[0], 1)[red[index].astype(np.bool8)]
        mask=np.isin(neigh, l, invert=True)
        return neigh[mask]
    
    def save(neigh, n_fam,  l, lll):
        l=np.append(l, neigh)
        lll[n_fam]+=neigh.size
        return l, lll
    
    def get_active_connected(red, i, l, lll, n_fam):
        new_neigh=get_new_neighbours(red, i, l)
        if new_neigh.size!=0:
            l,lll=save(new_neigh, n_fam, l, lll)
            for j in new_neigh:
                l, lll =get_active_connected(red, j, l, lll, n_fam)
        return l,lll
    
    for i in range(n_active):
        if i not in l:
            n_fam+=1
            l=np.append(l,np.array((i)))
            lll=np.append(lll, np.array((1)))
            l, lll =get_active_connected(red,i, l, lll, n_fam) 
    return np.flip(np.sort(lll))#sorted(lll, reverse=True)

In [8]:
%timeit get_families_py(red)
%timeit get_families_np(red)

355 µs ± 24.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
440 µs ± 6.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
